In [21]:
import paramiko
from mutagen.mp3 import MP3
from pydub import AudioSegment
import os

def extract_file_prefix(url):
    filename = url.split('/')[-1]
    return filename.split('_')[0]

def first_suffix_for_sound_directory(url):
    filename = url.split('/')[-2]
    return filename

hostname = 'files.clarksonmsda.org'
port = 2022
username = 'kadaliu@clarkson.edu'
password = 'Act.324fgdg'
remote_directory = '/srv/sftpgo/data/users/kadaliu@clarkson.edu/traffic/snd/'
local_image_dir = '/Users/tejeshchintpalli/Downloads/capstone/faultimages/'
local_audio_dir = '/Users/tejeshchintpalli/Downloads/capstone/faultaudio/'

def check_audio_file(file_path):
    file_size = os.path.getsize(file_path)
    print(f"File size: {file_size} bytes")
    audio = MP3(file_path)
    duration = audio.info.length
    print(f"Duration: {duration} seconds")
    audio_segment = AudioSegment.from_mp3(file_path)
    average_dbfs = audio_segment.dBFS
    print(f"Average audio level: {average_dbfs} dBFS")
    return file_size, duration, average_dbfs

try:
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(hostname, port, username, password)
    sftp = client.open_sftp()


    with open('/Users/tejeshchintpalli/Downloads/capstone/fulllogs1.txt', 'r') as file:
        urls = file.readlines()

    for url in urls:
        url = url.strip()  
        file_prefix = extract_file_prefix(url)
        file_suffix = first_suffix_for_sound_directory(url)
        updated_remote_directory = remote_directory + file_suffix + '/'

        files = sftp.listdir(updated_remote_directory)
        matching_files = [f for f in files if f.startswith(file_prefix)]

        img_filename = url.split('/')[-1]
        remote_img_path = url
        local_img_path = os.path.join(local_image_dir, img_filename)
        sftp.get(remote_img_path, local_img_path)
        print(f"Image file downloaded successfully to {local_img_path}")

        if not matching_files:
            print("No files found with the specified prefix")
        else:
            for file in matching_files:
                remote_path = os.path.join(updated_remote_directory, file)
                local_path = os.path.join(local_audio_dir if file.endswith('.mp3') else local_image_dir, file)
                sftp.get(remote_path, local_path)
                print(f"File downloaded successfully to {local_path}")

                if file.endswith('.mp3'):
                    file_size, duration, average_dbfs = check_audio_file(local_path)
                    if file_size > 0 and average_dbfs > -60:
                        print({"code": 0, "msg": "ok"})
                    else:
                        print({"code": 1, "msg": "Check Sound "})

    sftp.close()
    client.close()

except Exception as e:
    print(f"An error occurred: {e}")


Image file downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultimages/070035_15536.jpg
File downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultaudio/070035_200000_.mp3
File size: 41779 bytes
Duration: 5.16 seconds
Average audio level: -22.3606278521674 dBFS
{'code': 0, 'msg': 'ok'}
Image file downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultimages/070035_15537.jpg
File downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultaudio/070035_898000_.mp3
File size: 43124 bytes
Duration: 5.328 seconds
Average audio level: -22.53631931215857 dBFS
{'code': 0, 'msg': 'ok'}
Image file downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultimages/070050_15539.jpg
File downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultaudio/070050_300000_.mp3
File size: 41779 bytes
Duration: 5.16 seconds
Average audio level: -21.221249356255253 dBFS
{'code': 0, 'msg': 'ok'}
Image 

In [8]:
import paramiko
from mutagen.mp3 import MP3
from pydub import AudioSegment
import os

def extract_file_prefix(url):
    return url.split('/')[-1].split('_')[0]

def first_suffix_for_sound_directory(url):
    return url.split('/')[-2]

hostname = 'files.clarksonmsda.org'
port = 2022
username = 'kadaliu@clarkson.edu'
password = 'Act.324fgdg'
remote_directory = '/srv/sftpgo/data/users/kadaliu@clarkson.edu/traffic/snd/'
local_image_dir = '/Users/tejeshchintpalli/Downloads/capstone/faultimg/'
local_audio_dir = '/Users/tejeshchintpalli/Downloads/capstone/faultaud/'

def check_audio_file(file_path):
    if not os.path.exists(file_path):
        return None, None, "File does not exist"
    if os.path.getsize(file_path) == 0:
        return None, None, "File is empty"
    audio = MP3(file_path)
    duration = audio.info.length
    audio_segment = AudioSegment.from_mp3(file_path)
    average_dbfs = audio_segment.dBFS
    print(f"File size: {os.path.getsize(file_path)} bytes")
    print(f"Duration: {duration} seconds")
    print(f"Average audio level: {average_dbfs} dBFS")
    return os.path.getsize(file_path), duration, average_dbfs

client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname, port, username, password)
sftp = client.open_sftp()

try:
    with open('/Users/tejeshchintpalli/Downloads/capstone/fulllogs1.txt', 'r') as file:
        urls = file.readlines()
except IOError:
    print("Failed to read URLs from file")
    client.close()
    exit(1)

for url in urls:
    url = url.strip()
    img_filename = url.split('/')[-1]
    remote_img_path = url
    local_img_path = os.path.join(local_image_dir, img_filename)

    try:
        sftp.get(remote_img_path, local_img_path)
        print(f"Image file downloaded successfully to {local_img_path}")
    except IOError:
        print(f"Failed to download image file: {remote_img_path}")

    file_prefix = extract_file_prefix(url)
    file_suffix = first_suffix_for_sound_directory(url)
    updated_remote_directory = remote_directory + file_suffix + '/'

    try:
        files = sftp.listdir(updated_remote_directory)
    except IOError:
        print(f"Failed to list directory: {updated_remote_directory}")
        continue

    for file in files:
        if file.startswith(file_prefix) and file.endswith('.mp3'):
            remote_path = os.path.join(updated_remote_directory, file)
            local_path = os.path.join(local_audio_dir, file)
            try:
                sftp.get(remote_path, local_path)
                print(f"Audio file downloaded successfully to {local_path}")
                check_audio_file(local_path)
            except IOError:
                print(f"Failed to download audio file: {file}")

sftp.close()
client.close()


Image file downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultimg/070035_15536.jpg
Audio file downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultaud/070035_200000_.mp3
File size: 41779 bytes
Duration: 5.16 seconds
Average audio level: -22.3606278521674 dBFS
Image file downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultimg/070035_15537.jpg
Audio file downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultaud/070035_898000_.mp3
File size: 43124 bytes
Duration: 5.328 seconds
Average audio level: -22.53631931215857 dBFS
Image file downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultimg/070050_15539.jpg
Audio file downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultaud/070050_300000_.mp3
File size: 41779 bytes
Duration: 5.16 seconds
Average audio level: -21.221249356255253 dBFS
Image file downloaded successfully to /Users/tejeshchintpalli/Downloads/capsto

In [20]:
import librosa
import numpy as np
import os

def analyze_audio(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None)
        if len(y) == 0:
            print(f"{file_path}: Audio file is empty.")
            return False

        energy = np.sum(y**2) / len(y)
        zcr = np.mean(librosa.feature.zero_crossing_rate(y=y))
        mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr), axis=1)

        print(f"{file_path}: Energy={energy}, ZCR={zcr}, MFCC Mean={np.mean(mfcc)}")

        energy_threshold = 0.004
        zcr_threshold = 0.025
        mfcc_threshold = -1.2

        if energy > energy_threshold and zcr > zcr_threshold and np.mean(mfcc) > mfcc_threshold:
            return True
        else:
            return False

    except Exception as e:
        print(f"Failed to process {file_path}: {str(e)}")
        return False

def find_vehicle_sounds(directory):
    vehicle_sounds = []
    for file in os.listdir(directory):
        if file.endswith('.mp3'):
            file_path = os.path.join(directory, file)
            if analyze_audio(file_path):
                vehicle_sounds.append(file)
    return vehicle_sounds

audio_dir = '/Users/tejeshchintpalli/Downloads/capstone/faultaud'
vehicle_sounds = find_vehicle_sounds(audio_dir)

print(f"Vehicle sounds detected in: {vehicle_sounds}")


/Users/tejeshchintpalli/Downloads/capstone/faultaud/testwrong.mp3: Energy=1.5738796680531604e-05, ZCR=0.006205629644373673, MFCC Mean=-14.744272232055664
/Users/tejeshchintpalli/Downloads/capstone/faultaud/070035_898000_.mp3: Energy=0.005578948605445123, ZCR=0.0340293715417505, MFCC Mean=-0.8949750661849976
/Users/tejeshchintpalli/Downloads/capstone/faultaud/070145_510000_.mp3: Energy=0.0051960608021157685, ZCR=0.02834128163343558, MFCC Mean=0.23783203959465027
/Users/tejeshchintpalli/Downloads/capstone/faultaud/070051_200000_.mp3: Energy=0.007659188409646352, ZCR=0.02781071335758836, MFCC Mean=0.019258927553892136
/Users/tejeshchintpalli/Downloads/capstone/faultaud/070202_600000_.mp3: Energy=0.010640436603176978, ZCR=0.03302137040995976, MFCC Mean=-0.29805317521095276
/Users/tejeshchintpalli/Downloads/capstone/faultaud/070050_300000_.mp3: Energy=0.007550330956776937, ZCR=0.028662413916320167, MFCC Mean=-0.43336257338523865
/Users/tejeshchintpalli/Downloads/capstone/faultaud/070157_260